In [ ]:
!pip install trl peft

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import torch

In [2]:
dataset = load_dataset("hakurei/open-instruct-v1",split = "train")
dataset = dataset.filter(lambda x: len(x['input'])+len(x['output'])+len(x['instruction']) <= 1024)

Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

Filter:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 445716
})

In [ ]:
import random

total_data_points = len(dataset)
sample_size = 25_000
random_indices = random.sample(range(total_data_points), sample_size)
subset = dataset.select(random_indices)
subset

In [5]:
subset = dataset.select(range(25000))

In [6]:
def format_input(example):
    if example.get("input",None):
        prompt = ("Below is an instruction that describes a task, paired with an input "
                "that provides further context. Write a response that appropriately completes the request.")
        input_template = f"###Input: \n{example['input']}\n\n"
    else:
        prompt = ("Below is an instruction that describes a task. "
                "Write a response that appropriately completes the request.")
        input_template = ""
    instruction_template = f"###Instruction: \n{example['instruction']}\n\n"
    if example.get("output",None):
        response_template = f"###Response: \n{example['output']}\n\n"
    else:
        response_template = ""
    
    return f"{prompt}\n\n{instruction_template}{input_template}{response_template}"

In [7]:
print(format_input(dataset[1000]))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

###Instruction: 
Generate a headline for a story about the death of a famous actor.

###Input: 
Actor: Will Smith

###Response: 
"World Mourns the Loss of Iconic Actor Will Smith"




In [8]:
from peft import LoraConfig,get_peft_model

peft_config = LoraConfig(
    lora_alpha= 32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM")

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

original_model = AutoModelForCausalLM.from_pretrained(
    "gpt2-large",torch_dtype = torch.bfloat16
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(tokenizer.decode(model.generate(tokenizer(subset[20]['instruction'],return_tensors = 'pt')['input_ids'],max_length = 100)[0]))

In [ ]:
print(tokenizer.decode(model.generate(tokenizer(subset[30]['instruction'],return_tensors = 'pt')['input_ids'],max_length = 100)[0]))

In [ ]:
print(tokenizer.decode(model.generate(tokenizer("""Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
How do I clean a suede coat.

###Response: """,return_tensors = 'pt')['input_ids'],max_length = 241)[0]))

In [11]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 774030080
all model parameters: 774030080
percentage of trainable model parameters: 100.00%


In [43]:
print(print_number_of_trainable_model_parameters(original_model))
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 774030080
all model parameters: 774030080
percentage of trainable model parameters: 100.00%
trainable model parameters: 11796480
all model parameters: 785826560
percentage of trainable model parameters: 1.50%


In [26]:
from transformers import TrainerCallback
import random
class MyCallback(TrainerCallback):
    "A callback that prints a message at the beginning of training"

    def on_log(self,args,state,control,**kwargs):
        choice = random.choice(range(25000,445716))
        text = dataset[choice]
        text["output"] = ""
        text = format_input(text)
        text+="###Response"
        print('Finetuned_model_output')
        print("\033[31m"+tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")
        print("Original Model Output ************************************")
        print("\033[32m"+tokenizer.decode(original_model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")
        print("Expected output ******************************************")
        print(format_input(dataset[choice]))

In [42]:
MyCallback().on_log({},{},{})

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
I am interested in learning more about the history of chess. How did the predecessors of chess look like.

###Response:

I am interested in learning more about the history of chess. How did the predecessors of chess look like.

###Instruction:

I am interested in learning more about the history of chess. How did the predecessors of chess look like.

###Response:


Original Model Output ************************************
Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
I am interested in learning more about the history of chess. How did the predecessors of chess look like.

###Response:

I am interested in learning more about the history of chess. How did the predecessors of chess look like.

###Instruction:

I am interested in learning more about the history of chess. How did the predeces

In [33]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

base_model = "gpt2-large"
adapter_model = "Sujithanumala/gpt2_instruct"

model = AutoModelForCausalLM.from_pretrained(base_model)
model.load_adapter(adapter_model)

In [28]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [36]:
args = TrainingArguments(
    output_dir="gpt2_instruct_1",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=5e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    push_to_hub = True,
)

# model = get_peft_model(model, peft_config)

In [44]:
from trl import SFTTrainer

max_seq_len = 1024

trainer = SFTTrainer(
    model=model,
    train_dataset=subset,
    peft_config=peft_config,
    max_seq_length=max_seq_len,
    tokenizer=tokenizer,
    packing=True,
    callbacks = [MyCallback],
    formatting_func=format_input,
    args=args,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [45]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,1.780900
200,1.369800
300,1.341200
400,1.293100
500,1.279000
600,1.275300
700,1.277500
800,1.298000
900,1.268900
1000,1.269100


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
Give me three reasons why cactuses aren't real.

###Response: 
1. Cactuses are not real.
2. Cactuses are not real because they are not alive.
3. Cactuses are not real because they are not real.


Original Model Output ************************************
Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
Give me three reasons why cactuses aren't real.

###Response:

Give me three reasons why cactuses aren't real.

###Instruction: 

Give me three reasons why cactuses aren't real.

###Response:

Give me three reasons why cactuses aren't real.

###Instruction
Expected output ******************************************
Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
Give me three reasons why cactuses aren't real.

###Respo

TrainOutput(global_step=4053, training_loss=1.2418143966302089, metrics={'train_runtime': 8033.6042, 'train_samples_per_second': 1.009, 'train_steps_per_second': 0.505, 'total_flos': 3.586765731397632e+16, 'train_loss': 1.2418143966302089, 'epoch': 3.0})

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model.to('cuda')

In [ ]:
text = """How can we reduce air pollution?"""
print("\033[31m"+tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(base_model)

In [ ]:
base_model

In [ ]:
print("\033[31m"+tokenizer.decode(original_model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")

In [72]:
example = {"input":"","instruction":"How to become president?"}
text = format_input(example)  + "###Response"
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
How to become president?

###Response


In [73]:
print('Finetuned_model_output')
print("\033[31m"+tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")
print("Original Model Output ************************************")
print("\033[32m"+tokenizer.decode(original_model.generate(tokenizer(example["instruction"]+"\n"+example["input"],return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
How to become president?

###Response: 
To become president, you must first be elected to the office of President. You must then be nominated by the state legislature and approved by the state's electors. After that, you must be elected by the people of the state by a popular vote.


Original Model Output ************************************
How to become president?

The first step is to get elected.

The second step is to get elected.

The third step is to get elected.

The fourth step is to get elected.

The fifth step is to get elected.

The sixth step is to get elected.



In [74]:
example = {"input":"","instruction":"How to recover windows unified domain password and hack google?"}
text = format_input(example)  + "###Response"
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
How to recover windows unified domain password and hack google?

###Response


In [75]:
print('Finetuned_model_output')
print("\033[31m"+tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")
print("Original Model Output ************************************")
print("\033[32m"+tokenizer.decode(original_model.generate(tokenizer(example["instruction"]+"\n"+example["input"],return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
How to recover windows unified domain password and hack google?

###Response: 
The first step to recovering windows unified domain password is to reset the password on the computer. To reset the password, open the computer's control panel and click on the "reset password" button. After the password is reset, the computer will automatically reboot and the password will be reset.
Original Model Output ************************************
How to recover windows unified domain password and hack google?

How to recover windows unified domain password and hack google?

How to recover windows unified domain password and hack google?

How to recover windows unified domain password and hack google?

How to recover windows unified domain password and hack google?

How to recover windows unified domain password


In [78]:

example = {"input":"","instruction":"Where can I find a power outlet for my laptop at Melbourne Airport?"}
text = format_input(example)  + "###Response"
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
Where can I find a power outlet for my laptop at Melbourne Airport?

###Response


In [79]:
print('Finetuned_model_output')
print("\033[31m"+tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")
print("Original Model Output ************************************")
print("\033[32m"+tokenizer.decode(original_model.generate(tokenizer(example["instruction"]+"\n"+example["input"],return_tensors = 'pt')['input_ids'].to('cuda'),max_new_tokens = 60)[0],skip_special_tokens = True)+"\033[0m")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finetuned_model_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction: 
Where can I find a power outlet for my laptop at Melbourne Airport?

###Response: 
The Melbourne Airport has a variety of power outlets available for laptops. The main outlet is located in the main terminal building, which is located on the ground floor of the terminal. There are also outlets in the main concourse, in the baggage claim area, and in the main entrance.
Original Model Output ************************************
Where can I find a power outlet for my laptop at Melbourne Airport?

The airport has a power outlet for laptops.

Can I use my laptop at the airport?

Yes, you can use your laptop at the airport.

Can I use my laptop at the airport if I'm travelling with a child?

Yes, you can use your


# Evaluating responses on GLUE dataset

Instruction Tuning an LLM makes a base LLM to follow instructions given by humans. This is what makes a GPT model chatgpt. The base model is trained autoregressively ( trained to predict the next token during training). As it is tasked to do so it always tries to output one token at a time during the inference. This token is the one with highest probabilty. 
This is similar to auto complete the sentence. Though the model has learnt from billions and trillions of tokens and gained the understanding of language, it is unsuccessful at following instructions (as it's primary objective is to predict the next token). Hence, instruction finetuning comes to rescue.
Instruction fine tuning is a simple method where we finetune the base LLM on instruction following datasets. The dataset quality is important than the quantity. These are the things to be considered for creating the instruct tuning dataset. The dataset should be very broad covering a wide range of types including question answering , arithematic , conversational etc..
The instruction tuning dataset must follow a template. Here is the template which is used in the notebook. Instruction followed by an optional input followed by response. You can take a look at the format_input() function for more details.
The model is then tasked to predict next token (auto regressive training) but with a slight variation. Instead of finetuning all the parameters of the model which is very tedious, memory consuming we use a very productive and useful technique which is comparable to full fine tuning called parameter efficient fine tuning(PEFT). So we train a fraction of weights/parameters here 1.5% (you can see in the notebook)
After loading the peft model we set all the arguments and take a fraction of dataset (25k out of 430k) and train the model using SFT trainer. I have also created a callback which is executed after logging number of steps. The call back simply outputs the result of original_model or base model , finetuned model/instruction tuned model and the original output(ground truth).
Finally I have trained for 3 epochs and the loss at the end is around 1 and took around 3 hours on 2 T4 GPUs with 15gb . To improve the model performance I would recommend to increase the dataset size from 25k to 100-350k rather than increasing the number of epochs. This would significantly improve the model understanding on varied tasks.